In [1]:
import torch
import pandas as pd
import numpy as np

from torch_geometric.data import Data
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('data/yoochoose-clicks.dat', header=None)
df.columns=['session_id','timestamp','item_id','category']

buy_df = pd.read_csv('data/yoochoose-buys.dat', header=None)
buy_df.columns=['session_id','timestamp','item_id','price','quantity']

item_encoder = LabelEncoder()
df['item_id'] = item_encoder.fit_transform(df.item_id)
df.head()

,session_id,timestamp,item_id,category
0,1,2014-04-07T10:51:09.277Z,2053,0
1,1,2014-04-07T10:54:09.868Z,2052,0
2,1,2014-04-07T10:54:46.998Z,2054,0
3,1,2014-04-07T10:57:00.306Z,9876,0
4,2,2014-04-07T13:56:37.614Z,19448,0


In [3]:
#randomly sample a couple of them
sampled_session_id = np.random.choice(df.session_id.unique(), 1000000, replace=False)
df = df.loc[df.session_id.isin(sampled_session_id)]
df.nunique()

session_id    1000000
timestamp     3567674
item_id         35726
category          241
dtype: int64

In [4]:
df['label'] = df.session_id.isin(buy_df.session_id)
df.head()

,session_id,timestamp,item_id,category,label
36,12,2014-04-02T10:30:13.176Z,29310,0,True
37,12,2014-04-02T10:33:12.621Z,29310,0,True
101,28,2014-04-05T15:25:35.032Z,44335,0,False
102,28,2014-04-05T15:27:11.668Z,44335,0,False
103,28,2014-04-05T15:29:59.565Z,27021,0,False


In [6]:
from torch_geometric.data import InMemoryDataset
from tqdm import tqdm
from typing import Optional, Callable, Union, Tuple, List

class YooChooseBinaryDataset(InMemoryDataset):
    def __init__(self, root: Optional[str] = None, transform: Optional[Callable] = None, pre_transform: Optional[Callable] = None, pre_filter: Optional[Callable] = None):
        super().__init__(root, transform, pre_transform, pre_filter)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def raw_file_names(self) -> Union[str, List[str], Tuple]:
        return []

    @property
    def processed_file_names(self):
        return ['./yoochoose_click_binary_1M_sess.dataset']

    def download(self):
        pass

    def process(self):
        data_list = []

        grouped = df.groupby("session_id")
        for session_id, group in tqdm(grouped):
            sess_item_id = LabelEncoder().fit_transform(group.item_id)
            group = group.reset_index(drop=True)
            group['sess_item_id'] = sess_item_id
            node_features = group.loc[group.session_id==session_id,['sess_item_id','item_id']].sort_values('sess_item_id').item_id.drop_duplicates().values

            node_features = torch.LongTensor(node_features).unsqueeze(1)
            target_nodes = group.sess_item_id.values[1:]
            source_nodes = group.sess_item_id.values[:-1]

            edge_index = torch.tensor([source_nodes, target_nodes], dtype=torch.long)

            x = node_features
            y = torch.FloatTensor([group.label.values[0]])

            data = Data(x=x, edge_index=edge_index, y=y)
            data_list.append(data)

        data, slices = self.collate(data_list)
        torch.save((data, slices), self.processed_paths[0])

In [7]:
dataset = YooChooseBinaryDataset("./data")

Processing...
100%|██████████| 1000000/1000000 [20:57<00:00, 795.21it/s] 
Done!


In [8]:
dataset = dataset.shuffle()
train_dataset = dataset[:800000]
val_dataset = dataset[800000:900000]
test_dataset = dataset[900000:]
len(train_dataset), len(val_dataset), len(test_dataset)

(800000, 100000, 100000)

In [ ]:
from torch.nn import Sequential as Seq, Linear, ReLU
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops

class SAGEConv(MessagePassing):
    